### Betterway 38. 간단한 인터페이스의 경우 클래스 대신 함수를 받아라

- Python 내장 모듈을 사용 하는 도중 사용자가 전달 한 함수를 실행 하는 경우, 이 함수를 훅(hook)이라고 부른다. 예시는 아래 코드와 같은 경우이다.
    - 리스트를 원소의 길이를 기준으로 정렬 하고 싶다고 가정 하였을 때

In [2]:
names = ["소크라테스", "아르키메데스",
        "플라톤", "아리스토텔레스"]

print(f"before : {names}")

names.sort(key = len)

print(f"after : {names}")

before : ['소크라테스', '아르키메데스', '플라톤', '아리스토텔레스']
after : ['플라톤', '소크라테스', '아르키메데스', '아리스토텔레스']


존재 하지 않는 키에 접근 할 때 로그를 기록 하고, 디폴트 값으로 0을 반환 하여 없는 값을 트래킹 할 수 있는 코드를 작성 할 때

In [4]:
def log_missing() :
    print("키 추가됨")
    return 0

In [10]:
from collections import defaultdict

current = {"초록" : 12, "파랑" : 3 }
increments = [("빨강", 5), ("파랑", 17), ("주황", 9)]
result = defaultdict(log_missing, current)

print(f"이전 : {dict(result)}")
for key, amount in increments : 
    result[key] += amount
print(f"이후 : {dict(result)}")

이전 : {'초록': 12, '파랑': 3}
키 추가됨
키 추가됨
이후 : {'초록': 12, '파랑': 20, '빨강': 5, '주황': 9}


딕셔너리에 없는 값에 대한 접근한 횟수를 카운트 하고 싶을 때 코드를 작성 해 보자.<br>
이 코드를 작성 할 때 접근 할 수 있는 방법 중 하나는 클로저가 있는 함수를 이용 하여 훅으로 사용 하는 것이다.

In [11]:
def increment_with_report(current, increments) :
    added_count = 0

    def missing() :
        nonlocal added_count
        added_count += 1
        return 0
        
    result = defaultdict(missing, current)
    for key, amount in increments :
        result[key] += amount
    
    return result, added_count

In [18]:
result, count = increment_with_report(current, increments)
assert count == 2

In [17]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local() 
    print("After local assignment:", spam) # 지역에 또 다른 spam 함수 생성이라, scope_test에 선언 된 변수 호출
    do_nonlocal()
    print("After nonlocal assignment:", spam) # 비지역 변수 이기 때문에 바깥 함수에 선언 된 test spam의 주소를 변경
    do_global()
    print("After global assignment:", spam) # global, 함수 바깥에 있는 값을 찾고 없기 때문에 변경 된 non local에 의해 변경 된 spam 호출

scope_test() 
print("In global scope:", spam) # 모든 함수 내부에서 메모리에 할당 된 변수가 사라지기 때문에 글로벌 스팸을 호출하고, 스팸 내용 변경한 값을 호출

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


- 다른 방법 중 클로저를 사용하는 방법이 아닌, 작은 클래스를 새롭게 정의 하는 것이다.


In [21]:
class CountMissing :
    def __init__(self) : 
        self.added = 0 
    
    def missing(self) :
        self.added += 1 
        return 0
        

In [22]:
counter = CountMissing()

result = defaultdict(counter.missing, current)
for key, amount in increments :
    result[key] += amount

assert counter.added == 2

- 이 클래스는 위 와 같은 클로저 보다는 가독성이 좋은 코드이다.

    - 하지만, 클래스 자체를 두고 보면 의미를 파악 하기 어렵다.
    
    - 이런 경우, 더 명확하게 표현 하기 위해서는 클래스에 `__call__` 특별 메서드를 정의 하면 사용 할 수 있다.

In [26]:
class BetterCountMissing :
    def __init__(self) :
        self.added = 0

    def __call__(self) :
        self.added += 1
        return 0
    
counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

print(counter())
print(callable(counter))

0
True


In [28]:
counter = BetterCountMissing()
result = defaultdict(counter, current)
for key, amount in increments :
    result[key] += amount
assert counter.added == 2
print(counter.added)

2
